In [175]:
import requests
import json
import time

In [176]:
# !pip3 install names-dataset

In [177]:
from names_dataset import NameDataset, NameWrapper

In [178]:
api_key = "usun_poczatek16831ee8bemshcddf2990350ca81p1d6a5fjsnbc1bd82e3091"

In [179]:
nd = NameDataset() 

In [180]:
name_value = 4
sleep_time = 5 

In [181]:
top_names_pl = nd.get_top_names(n=name_value, gender='Male', country_alpha2='PL')
print(top_names_pl)
print(top_names_pl["PL"]["M"])

{'PL': {'M': ['Piotr', 'Marcin', 'Krzysztof', 'Paweł']}}
['Piotr', 'Marcin', 'Krzysztof', 'Paweł']


In [182]:
top_names_pl["PL"]["F"] = nd.get_top_names(n=name_value, gender='Female', country_alpha2='PL')["PL"]["F"]
print(top_names_pl)
print(top_names_pl["PL"]["F"])

{'PL': {'M': ['Piotr', 'Marcin', 'Krzysztof', 'Paweł'], 'F': ['Anna', 'Agnieszka', 'Katarzyna', 'Monika']}}
['Anna', 'Agnieszka', 'Katarzyna', 'Monika']


In [183]:
top_names = top_names_pl.copy()

In [184]:
top_names

{'PL': {'M': ['Piotr', 'Marcin', 'Krzysztof', 'Paweł'],
  'F': ['Anna', 'Agnieszka', 'Katarzyna', 'Monika']}}

In [185]:
!ls

 backup			      data_prepare.ipynb   voivodeships0.json
'data_prepare (copy).ipynb'   regions.json	   voivodeships2.json


In [186]:
# user data download 

In [187]:
# load country data 
try:
    with open('regions.json', 'r') as voivodeships:
        voivodeships_data = json.load(voivodeships)
except Exception as e:
    print(e)

In [188]:
voivodeships_data

{'PL': [{'name': 'Mazowsze', 'cities': ['warsaw', 'lodz']},
  {'name': 'Wielkopolska', 'cities': ['poznan', 'szczecin']},
  {'name': 'Małopolska', 'cities': ['krakow', 'rzeszow']},
  {'name': 'Śląsk', 'cities': ['katowice', 'gliwice']},
  {'name': 'Pomorze', 'cities': ['gdansk', 'gdynia']}],
 'UK': [{'name': 'England', 'cities': ['london', 'birmingham']},
  {'name': 'Scotland', 'cities': ['glasgow', 'edinburgh']},
  {'name': 'Wales', 'cities': ['cardiff', 'swansea']},
  {'name': 'Northern Ireland', 'cities': ['belfast', 'londonderry']}]}

In [189]:
users = {}

In [190]:
for country_key in list(voivodeships_data.keys()):
    users.update({country_key : {} })
    for voivodeship in voivodeships_data[country_key]:
        users[country_key].update({voivodeship["name"] : {}})
        for city in voivodeship["cities"]:
            users[country_key][voivodeship["name"]].update({city :[]})   

In [191]:
users

{'PL': {'Mazowsze': {'warsaw': [], 'lodz': []},
  'Wielkopolska': {'poznan': [], 'szczecin': []},
  'Małopolska': {'krakow': [], 'rzeszow': []},
  'Śląsk': {'katowice': [], 'gliwice': []},
  'Pomorze': {'gdansk': [], 'gdynia': []}},
 'UK': {'England': {'london': [], 'birmingham': []},
  'Scotland': {'glasgow': [], 'edinburgh': []},
  'Wales': {'cardiff': [], 'swansea': []},
  'Northern Ireland': {'belfast': [], 'londonderry': []}}}

In [192]:
def get_users(city, name, offset=None, api_key=api_key):
    url = "https://soundcloud-scraper.p.rapidapi.com/v1/search/users"

    if not offset:
        querystring = {"term":name,"place":city}
    else:
        querystring = {"term":name,"offset":offset,"place":city}
    
    headers = {
    	"X-RapidAPI-Key": api_key,
    	"X-RapidAPI-Host": "soundcloud-scraper.p.rapidapi.com"
    }
    
    response = requests.get(url, headers=headers, params=querystring)
    
    return response.json()

In [193]:
def get_users_for_country(users, country, top_names, offset=None, api_key=api_key): 
    i = 0
    names = top_names[country]['M'] + top_names[country]['F']
    for voivodeship in users[country].keys():
        for city in users[country][voivodeship].keys():
            for name in names:
                print(f"looking for users with name {name} in {city}")
                users_result = get_users(city, name)
                cnt = users_result['totalCount']
                print(f"got {cnt if cnt <= 50 else 50} of maximum {cnt} possible" )
                users[country][voivodeship][city] += users_result["users"]["items"]
                save_to_file(f"users_{name}_{city}_{country}", users)    
                i += 1
                if i >= 2:
                    break
                time.sleep(sleep_time)
        
    return users

In [194]:
users = get_users_for_country(users, 'PL', top_names)

looking for users with name Piotr in warsaw
got 50 of maximum 412 possible
looking for users with name Marcin in warsaw
got 50 of maximum 392 possible
looking for users with name Piotr in lodz
got 50 of maximum 120 possible
looking for users with name Piotr in poznan
got 50 of maximum 176 possible
looking for users with name Piotr in szczecin
got 50 of maximum 81 possible
looking for users with name Piotr in krakow
got 50 of maximum 124 possible
looking for users with name Piotr in rzeszow
got 25 of maximum 25 possible
looking for users with name Piotr in katowice
got 50 of maximum 59 possible
looking for users with name Piotr in gliwice
got 23 of maximum 23 possible
looking for users with name Piotr in gdansk
got 50 of maximum 101 possible
looking for users with name Piotr in gdynia
got 50 of maximum 51 possible


In [195]:
users

{'PL': {'Mazowsze': {'warsaw': [{'type': 'user',
     'id': 133933703,
     'permalink': 'https://soundcloud.com/piotr-ho',
     'createdAt': '2015-01-19T20:39:26Z',
     'lastModified': '2023-11-16T19:28:13Z',
     'name': 'Piotr Ho',
     'description': 'contact: piotr.horelik@gmail.com\n\nPiotr Ho is a Warsaw DJ and promoter active on the scene since 2016. Organization of events integrating various genres, from downtempo, through house and techno, to psychedelic trance music, confirmed his need for a cross-sectional view on music. It resulted in regular performances at various events, where he presents his favorite sounds selection.\nThe mystical atmosphere of the community, full of symbiosis between the artist and the dance floor, as well as the infinity of new "forms" in electronic music, is the basis for the musical existence of the Warsaw artist. Superiority of a deep groove over the pace with a rough kick, is a principle he follows to keep the hypnotic atmosphere. His favourite

In [196]:
save_to_file("saved_users", users)